In [1]:
import pandas as pd
import numpy as np
import duckdb # библиотека для работы с SQL

In [2]:
users = pd.read_csv('users.csv')
sessions_closed = pd.read_csv('sessions_closed.csv')

#### Это задание1 сделанное полностью на SQL. Создам нескольк SQL запросов

In [3]:
# в этом запросе я произвожу группировку и заполняю значения null
# role заполняю значениямии full, так как это самый частовстречаемый тип роли
# registration_date заполненяю первой датой создания задания

query01 = '''
    SELECT 
        sc.user_id,
        COALESCE(u.role, 'full') as role, 
        MIN(COALESCE(strftime('%Y-%m-%d', STRPTIME(u.create_at, '%d/%m/%y %H:%M'))::date, sc.created_at::date)) as registration_date
        --sc.id as session_id,
        --sc.status,
        --sc.created_at::date as created_at,
        --sc.updated_at::date as updated_at,
        --EXTRACT('year' FROM DATE sc.updated_at::date)
        FROM sessions_closed sc
    LEFT JOIN users u ON u.user_id = sc.user_id
    GROUP BY sc.user_id, role
'''

In [4]:
df_query01 = duckdb.query(query01).to_df()

In [5]:
df_query01.shape

(59456, 3)

In [6]:
df_query01

,user_id,role,registration_date
0,52626,full,2022-08-18
1,208847,full,2023-11-21
2,27901,full,2022-05-23
3,124600,full,2023-03-19
4,172768,full,2023-09-04
...,...,...,...
59451,25842,full,2022-05-16
59452,25863,full,2022-05-16
59453,25994,full,2022-05-18
59454,25897,full,2022-05-17


In [7]:
# в этом запросе я считаю дату последней активности
# 100, 102 - дата последней активности = created_at
# 101, 103 - дата последней активности = updated_at

query02 = '''
    SELECT 
        sc.user_id,
        MAX(CASE
            WHEN status IN (100, 102) THEN sc.created_at::date
            WHEN status IN (101, 103) THEN sc.updated_at::date
            ELSE null
        END) as last_activity_date
    FROM sessions_closed sc
    LEFT JOIN users u ON u.user_id = sc.user_id
    GROUP BY sc.user_id
'''

In [8]:
df_query02 = duckdb.query(query02).to_df()

In [9]:
df_query02.shape

(59456, 2)

In [10]:
df_query02

,user_id,last_activity_date
0,55396,2023-11-30
1,92027,2024-01-12
2,72893,2023-11-30
3,213484,2023-12-03
4,58998,2023-12-25
...,...,...
59451,23702,2022-10-24
59452,75553,2022-10-24
59453,70955,2022-10-27
59454,16500,2022-10-28


In [11]:
# в этом запросе считаю количество успешных выполнений в разрезе года
# создал несколько cte, где будет считать количество успешных заданий за каждый код

query03 = '''
WITH cte2021 AS
(
    SELECT 
        sc.user_id,
        COUNT(status) FILTER(sc.status = 103 AND EXTRACT('year' FROM sc.updated_at::date) = 2021) as 'y2021'
        FROM sessions_closed sc
    LEFT JOIN users u ON u.user_id = sc.user_id
    GROUP BY sc.user_id
),
cte2022 AS
(
    SELECT 
        sc.user_id,
        COUNT(status) FILTER(sc.status = 103 AND EXTRACT('year' FROM sc.updated_at::date) = 2022) as 'y2022'
        FROM sessions_closed sc
    LEFT JOIN users u ON u.user_id = sc.user_id
    GROUP BY sc.user_id
),
cte2023 AS
(
    SELECT 
        sc.user_id,
        COUNT(status) FILTER(sc.status = 103 AND EXTRACT('year' FROM sc.updated_at::date) = 2023) as 'y2023'
        FROM sessions_closed sc
    LEFT JOIN users u ON u.user_id = sc.user_id
    GROUP BY sc.user_id
),
cte2024 AS
(
    SELECT 
        sc.user_id,
        COUNT(status) FILTER(sc.status = 103 AND EXTRACT('year' FROM sc.updated_at::date) = 2024) as 'y2024'
        FROM sessions_closed sc
    LEFT JOIN users u ON u.user_id = sc.user_id
    GROUP BY sc.user_id
)

SELECT cte2021.user_id, y2021, y2022, y2023, y2024 FROM cte2021
LEFT JOIN cte2022 using(user_id)
LEFT JOIN cte2023 using(user_id)
LEFT JOIN cte2024 using(user_id)

'''

In [12]:
df_query03 = duckdb.query(query03).to_df()

In [13]:
df_query03.shape

(59456, 5)

In [14]:
df_query03

,user_id,y2021,y2022,y2023,y2024
0,213367,0,0,0,0
1,52626,0,1,5,0
2,166655,0,0,4,0
3,194920,0,0,0,0
4,3758,1,87,4,0
...,...,...,...,...,...
59451,213267,0,0,1,0
59452,213273,0,0,1,0
59453,213312,0,0,1,0
59454,44682,0,0,1,0


In [15]:
# соберу все запросы воедино

main_query = '''
    SELECT * FROM df_query01
    LEFT JOIN df_query02 using(user_id)
    LEFT JOIN df_query03 using(user_id)
'''

In [16]:
df_main_query = duckdb.query(main_query).to_df()

In [17]:
df_main_query.sample(10)

,user_id,role,registration_date,last_activity_date,y2021,y2022,y2023,y2024
9785,28594,full,2022-05-24,2022-05-26,0,1,0,0
29768,362,full,2021-12-06,2023-12-09,0,1,13,0
7675,57029,full,2022-09-02,2023-12-10,0,1,1,0
16761,51277,full,2022-08-16,2023-04-01,0,0,5,0
39357,3737,full,2021-12-21,2022-06-17,0,1,0,0
6544,88537,full,2022-11-11,2022-11-11,0,1,0,0
24516,74987,full,2022-10-22,2022-10-22,0,1,0,0
20461,11029,full,2022-02-06,2023-11-15,0,0,0,0
34764,90372,full,2022-11-18,2023-09-19,0,0,0,0
14496,8134,full,2022-01-23,2022-01-23,0,1,0,0


In [18]:
df_main_query.sort_values('y2021')

,user_id,role,registration_date,last_activity_date,y2021,y2022,y2023,y2024
0,52626,full,2022-08-18,2023-12-26,0,1,5,0
39328,11387,full,2022-02-07,2022-05-23,0,1,0,0
39329,2415,full,2021-12-07,2023-01-19,0,0,2,0
39330,27955,full,2022-05-23,2022-05-23,0,0,0,0
39331,28038,full,2022-05-24,2022-08-15,0,1,0,0
...,...,...,...,...,...,...,...,...
1020,1519,full,2021-12-10,2023-03-02,86,177,3,0
38162,254,full,2021-12-06,2023-10-20,87,353,123,0
22359,1019,full,2021-12-06,2023-12-28,89,629,407,0
22603,2228,full,2021-12-07,2023-12-07,93,261,22,0


In [20]:
df_main_query.to_csv('task_01_sql_result.csv')